# About this notebook

Demonstrate:
* Code example for Semantic Search 
* Code example for Generative Search

See [readme.md](./readme.md) for configuration instructions.

In [ ]:
!pip install weaviate-client -U

## Query data

In [ ]:
import os
import weaviate

auth_config = weaviate.AuthBearerToken(
  access_token = os.getenv("COHERE_AUTH_READ"),
  expires_in=300, # in seconds, by default 60s
)

client = weaviate.Client(
    url="https://cohere-demo.weaviate.network",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    },
    timeout_config = (20, 240),
)
client.is_ready()

In [169]:
def semantic_serch(query):
    nearText = {
        "concepts": [query]
    }

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get("Wikipedia", properties)
        .with_near_text(nearText)
        # .with_where({
        #     "path": ["lang"],
        #     "operator": "Equal",
        #     "valueText": "en"
        # })
        .with_limit(20)
        .do()
    )

    return response['data']['Get']['Wikipedia']


def print_result(result):
    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

In [ ]:
# Pass the user query to Weaviate
query_result = semantic_serch("సంగీత వాయిద్యాలు")

# Print out the result
print_result(query_result)

In [157]:
def generative_serch(query):

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get('Wikipedia', properties)
        .with_near_text({'concepts': [query]})
        .with_limit(2)
        .with_generate(single_prompt='write a poem based on: {text}')
        .do()
    )

    return response['data']['Get']['Wikipedia']

def print_gen_result(result):
    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['_additional']['generate']['singleResult']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

In [ ]:
# Pass the user query to Weaviate
query_result = generative_serch("how planes fly")

# Print out the result
print(query_result)
print_gen_result(query_result)

In [ ]:
# Test that all data has loaded – get object count
result = (
    client.query.aggregate("Wikipedia")
    .with_meta_count()
    .do()
)
print("Object count: ", result["data"]["Aggregate"]["Wikipedia"])

In [ ]:
client.get_meta()